Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import json

In [4]:
import os

In [5]:
from tqdm import tqdm

In [6]:
import multiprocessing

In [7]:
from functools import partial

In [8]:
from time import sleep

In [9]:
n_cpus = multiprocessing.cpu_count()

# Trying data from art institute of Chicago

In [13]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36" ,
    'referer':'https://www.google.co.in/'
}

In [16]:
def single_process(pid):
    with open("all_artworks_api_response_data_{}.jsonl".format(pid), 'r') as json_file:
        all_artworks_json = list(json_file)

    fo = open("artwork_description_{}.jsonl".format(pid), "w")
    fo.close()
    counter = 0
    for artwork in tqdm(all_artworks_json):
        try:
            art_id = json.loads(artwork)["data"]["id"]
        except:
            print(json.loads(artwork))
        url_template = "https://www.artic.edu/artworks/{}"
        try:
            res = requests.get(url_template.format(art_id), headers=headers)
            sleep(1)
            soup = BeautifulSoup(res.content)
        except:
            sleep(30)
            res = requests.get(url_template.format(art_id), headers=headers)
            continue
        try:
            description = soup.find_all("div", {"class":"o-blocks", "itemprop":"description"})[0]
        except:
            # raise "Description not available"
            continue
        try:
            description = description.find_all("p")
        except:
            print(description)
            raise "Something unexpected in description"

        with open("artwork_description_{}.jsonl".format(pid), "a") as fo:
            fo.write("{"+f'"art_id":{art_id}, "description":{[i.contents for i in description]}'+"}")
            fo.write("\n")


The website doesn't allow parallel process from same ip to make requests, it starts giving 403 error.
It also needs 1 second delay between every request.
The art id data can be split up into chunks and each chunk can be used by a different cloud instance if you want to execute in parallel.

In [ ]:
# parallel_processes = []
# for i in range(n_cpus):
#     p = multiprocessing.Process(target=partial(single_process, i))
#     parallel_processes.append(p)
# for proc in parallel_processes:
#     proc.start()

In [ ]:
single_process(0)

In [ ]:
single_process(1)

In [ ]:
single_process(2)

In [ ]:
single_process(3)